## Overview
The business has been investing in four different display advertising campaigns. You have been asked to evaluate how effective
each advertising campaign is in generating sales. Please see below the explanation of the data and the business questions.

The data on the left include information on 10000 consumers who clicked on at least one of the display ads from campaigns
A, B, C, or D. Purchase is indicated by the "Conversion" variable (i.e., equals 1 if there is purchase and 0 otherwise). The "Value" column indicates the revenue in dollars earned from each purchase. The cost per click is $7, $5, $4, and $2 for campaign A, B, C, and D, respectively. The order of clicks is as indicated in the data.

You are asked to build a statistical model (so not a rule based attribution model such as last click attribution) and answer:

Questions
1. Which campaign is the most successful in terms of unit sales contributed?

2. What is the return on investment for each campaign?

3. How would you optimize the spend of a given budget of $1 million across all four campaigns?

#### Assume the cost of campaign is only charged when there is a conversion
#### The idea is to construct transition probability matrix between campaigns, to evaluate each campaign effect on conversion units

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from collections import defaultdict

from pprint import pprint
from hmmlearn import hmm
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("./data/user_click.csv")
columns = ["user", "click_1", "click_2", "click_3", "click_4", "click_5", "conversion", "value"]
df = df[columns]
display(df.head(5))

,user,click_1,click_2,click_3,click_4,click_5,conversion,value
0,1,d,d,a,b,NaN,0,0
1,2,c,NaN,NaN,NaN,NaN,0,0
2,3,b,d,a,b,c,1,44
3,4,b,a,a,NaN,NaN,0,0
4,5,d,b,d,NaN,NaN,0,0


In [ ]:
# Step 1: Data Preprocessing
data = df.copy()
campaign_mapping = {'a': 0, 'b': 1, 'c': 2, 'd': 3}
data.replace(campaign_mapping, inplace=True)
data.fillna(-1, inplace=True)

# Step 2: Define the HMM
n_components = 2
model = hmm.MultinomialHMM(n_components=n_components)

# Step 3: Train the HMM
X = data[['click_1', 'click_2', 'click_3', 'click_4', 'click_5']].values
model.fit(X)

# Step 4: Attribution Modeling
def removal_effect(clickstream, campaign):
    clickstream = clickstream.copy()
    clickstream[clickstream == campaign] = -1
    without_campaign = model.predict_proba(clickstream.reshape(1, -1))[0][-1]
    with_campaign = model.predict_proba(clickstream.reshape(1, -1))[0][-1]
    return with_campaign - without_campaign

campaign_contributions = {campaign: 0 for campaign in campaign_mapping.values()}
for _, row in data.iterrows():
    clickstream = row[['click_1', 'click_2', 'click_3', 'click_4', 'click_5']].values
    for campaign in campaign_mapping.values():
        if campaign in clickstream:
            campaign_contributions[campaign] += removal_effect(clickstream, campaign)

# Step 5: ROI Calculation
cost_per_click = {'a': 7, 'b': 5, 'c': 4, 'd': 2}
campaign_costs = {campaign: data[f'click_{i+1}'].value_counts().get(campaign, 0) * cost_per_click[campaign_label]
                  for i, (campaign_label, campaign) in enumerate(campaign_mapping.items())}
campaign_revenues = {campaign: campaign_contributions[campaign] * data['Value'].sum() for campaign in campaign_mapping.values()}
campaign_roi = {campaign: (revenue - cost) / cost for campaign, revenue, cost in zip(campaign_mapping.values(), campaign_revenues.values(), campaign_costs.values())}

# Step 6: Budget Optimization
budget = 1000000
# Use an optimization algorithm (e.g., linear programming) to allocate the budget based on ROI and constraints
# Implement the optimization algorithm here

# Print the results
print("Campaign Contributions:")
for campaign, contribution in campaign_contributions.items():
    print(f"Campaign {list(campaign_mapping.keys())[campaign]}: {contribution}")

print("\nROI:")
for campaign, roi in campaign_roi.items():
    print(f"Campaign {list(campaign_mapping.keys())[campaign]}: {roi}")

print("\nOptimized Budget Allocation:")
# Print the optimized budget allocation here

In [3]:
df[["click_1", "click_2", "click_3", "click_4", "click_5"]] = df[["click_1", "click_2", "click_3", "click_4", "click_5"]].fillna("NULL")

#### DEFINE STATES
##### START, a, b, c, d, NULL, Conversion

In [4]:
#The cost per click is $7, $5, $4, and $2 for campaign A, B, C, and D, respectively. 
# The company pay for the cost per click
# the profit = revenue 
# cpc_dict = {"a": 7, "b": 5, "c": 4, "d": 2}
# df["cost"] = 0
# for i in range(1, 6):
#     df[f"click_{i}_cost"] = df[f"click_{i}"].map(cpc_dict).fillna(0)
#     df["cost"] += df[f"click_{i}_cost"]
# df["profit"] = df["value"] - df["cost"]
# df.profit[df.profit < 0] = 0
df["path"] = df[["click_1", "click_2", "click_3", "click_4", "click_5"]].astype(str).agg(" ".join, axis=1) #.str.split(" ")

In [5]:
df['path'] = np.where(df['conversion'] == 0,
                      ['Start '] + df['path'] + [' NULL'],
                      ['Start '] + df['path'] + [' Conversion'])
df["path"] = df["path"].str.split(" ")

In [6]:
df.head(5)

,user,click_1,click_2,click_3,click_4,click_5,conversion,value,path
0,1,d,d,a,b,NULL,0,0,"[Start, d, d, a, b, NULL, NULL]"
1,2,c,NULL,NULL,NULL,NULL,0,0,"[Start, c, NULL, NULL, NULL, NULL, NULL]"
2,3,b,d,a,b,c,1,44,"[Start, b, d, a, b, c, Conversion]"
3,4,b,a,a,NULL,NULL,0,0,"[Start, b, a, a, NULL, NULL, NULL]"
4,5,d,b,d,NULL,NULL,0,0,"[Start, d, b, d, NULL, NULL, NULL]"


In [7]:
list_of_paths = df['path']
total_conversions = sum(path.count('Conversion') for path in df['path'].tolist())
base_conversion_rate = total_conversions / len(list_of_paths)
total_value = df["value"].sum()
base_revenue_rate = total_value / len(list_of_paths)
# total_profit = df["profit"].sum()
# base_profit_rate = total_profit / len(list_of_paths)

#### BUILD TRANSITION STATES

In [8]:
def transition_states(list_of_paths):
    list_of_unique_channels = set(x for element in list_of_paths for x in element)
    #print(list_of_unique_channels)
    transition_states = {x + '-->' + y: 0 for x in list_of_unique_channels for y in list_of_unique_channels}
    #print(transition_states)
    for possible_state in list_of_unique_channels:
        if possible_state not in ['Conversion', 'NULL']:
            for user_path in list_of_paths:
                if possible_state in user_path:
                    indices = [i for i, s in enumerate(user_path) if possible_state in s]
                    for col in indices:
                        #print(col)
                        #print(user_path)
                        #print(user_path[col])
                        if col < len(user_path) - 1:
                            transition_states[user_path[col] + '-->' + user_path[col + 1]] += 1

    return transition_states

In [9]:
trans_states = transition_states(list_of_paths)

In [10]:
trans_states

{'d-->d': 1223,
 'd-->a': 1219,
 'd-->c': 1212,
 'd-->b': 1236,
 'd-->Start': 0,
 'd-->NULL': 2334,
 'd-->Conversion': 157,
 'a-->d': 1214,
 'a-->a': 1226,
 'a-->c': 1212,
 'a-->b': 1286,
 'a-->Start': 0,
 'a-->NULL': 2243,
 'a-->Conversion': 183,
 'c-->d': 1228,
 'c-->a': 1168,
 'c-->c': 1256,
 'c-->b': 1256,
 'c-->Start': 0,
 'c-->NULL': 2356,
 'c-->Conversion': 165,
 'b-->d': 1259,
 'b-->a': 1258,
 'b-->c': 1252,
 'b-->b': 1277,
 'b-->Start': 0,
 'b-->NULL': 2370,
 'b-->Conversion': 192,
 'Start-->d': 3403,
 'Start-->a': 4986,
 'Start-->c': 3426,
 'Start-->b': 3513,
 'Start-->Start': 0,
 'Start-->NULL': 0,
 'Start-->Conversion': 0,
 'NULL-->d': 0,
 'NULL-->a': 0,
 'NULL-->c': 0,
 'NULL-->b': 0,
 'NULL-->Start': 0,
 'NULL-->NULL': 0,
 'NULL-->Conversion': 0,
 'Conversion-->d': 0,
 'Conversion-->a': 0,
 'Conversion-->c': 0,
 'Conversion-->b': 0,
 'Conversion-->Start': 0,
 'Conversion-->NULL': 0,
 'Conversion-->Conversion': 0}

#### FROM TRANSITION FREQUENCY, BUILD TRANSITION PROBABILITY MATRIX

In [11]:
def transition_probability(trans_dict):
    list_of_unique_channels = set(x for element in list_of_paths for x in element)
    trans_prob = defaultdict(dict)
    for state in list_of_unique_channels:
        if state not in ['Conversion', 'NULL']:
            counter = 0
            index = [i for i, s in enumerate(trans_dict) if state + '-->' in s]
            for col in index:
                if trans_dict[list(trans_dict)[col]] > 0:
                    counter += trans_dict[list(trans_dict)[col]]
            for col in index:
                if trans_dict[list(trans_dict)[col]] > 0:
                    state_prob = float((trans_dict[list(trans_dict)[col]])) / float(counter)
                    trans_prob[list(trans_dict)[col]] = state_prob

    return trans_prob

In [12]:
state_probability = transition_probability(trans_states)
pprint(state_probability)

defaultdict(<class 'dict'>,
            {'Start-->a': 0.32528705636743216,
             'Start-->b': 0.2291884133611691,
             'Start-->c': 0.2235125260960334,
             'Start-->d': 0.22201200417536535,
             'a-->Conversion': 0.024850624660510592,
             'a-->NULL': 0.30458989679521997,
             'a-->a': 0.16648560564910375,
             'a-->b': 0.1746333514394351,
             'a-->c': 0.1645844649646931,
             'a-->d': 0.1648560564910375,
             'b-->Conversion': 0.025236593059936908,
             'b-->NULL': 0.3115141955835962,
             'b-->a': 0.16535226077812828,
             'b-->b': 0.16784963196635122,
             'b-->c': 0.16456361724500526,
             'b-->d': 0.16548370136698212,
             'c-->Conversion': 0.02221025710055189,
             'c-->NULL': 0.3171355498721228,
             'c-->a': 0.15722169874814915,
             'c-->b': 0.1690671692017768,
             'c-->c': 0.1690671692017768,
             'c-->d': 0.

In [13]:
def construct_transition_matrix(list_of_paths, state_probability):
    transition_matrix = pd.DataFrame()
    list_of_unique_channels = set(x for element in list_of_paths for x in element)

    for channel in list_of_unique_channels:
        transition_matrix[channel] = 0.00
        transition_matrix.loc[channel] = 0.00
        transition_matrix.loc[channel][channel] = 1.0 if channel in ['Conversion', 'NULL'] else 0.0

    for key, value in state_probability.items():
        origin, destination = key.split('-->')
        transition_matrix.at[origin, destination] = value

    return transition_matrix

transition_matrix = construct_transition_matrix(list_of_paths, state_probability)
print("Markov transition matrix")
display(transition_matrix)

Markov transition matrix


,d,a,c,b,Start,NULL,Conversion
d,0.165696,0.165154,0.164205,0.167457,0.0,0.316217,0.021271
a,0.164856,0.166486,0.164584,0.174633,0.0,0.304590,0.024851
c,0.165298,0.157222,0.169067,0.169067,0.0,0.317136,0.022210
b,0.165484,0.165352,0.164564,0.167850,0.0,0.311514,0.025237
Start,0.222012,0.325287,0.223513,0.229188,0.0,0.000000,0.000000
NULL,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
Conversion,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


#### HAVING TRANSITION MATRIX, CALCULATE REMOVAL EFFECTS OF EACH CAMPAIGN

In [14]:
def removal_effects(df, conversion_rate):
    removal_effects_dict = {}
    channels = [channel for channel in df.columns if channel not in ['Start',
                                                                     'NULL',
                                                                     'Conversion']]
    for channel in channels:
        removal_df = df.drop(channel, axis=1).drop(channel, axis=0)
        for column in removal_df.columns:
            row_sum = np.sum(list(removal_df.loc[column]))
            null_pct = float(1) - row_sum
            if null_pct != 0:
                removal_df.loc[column]['NULL'] = null_pct
            removal_df.loc['NULL']['NULL'] = 1.0

        removal_to_conv = removal_df[
            ['NULL', 'Conversion']].drop(['NULL', 'Conversion'], axis=0)
        removal_to_non_conv = removal_df.drop(
            ['NULL', 'Conversion'], axis=1).drop(['NULL', 'Conversion'], axis=0)

        removal_inv_diff = np.linalg.inv(
            np.identity(
                len(removal_to_non_conv.columns)) - np.asarray(removal_to_non_conv))
        removal_dot_prod = np.dot(removal_inv_diff, np.asarray(removal_to_conv))
        removal_cvr = pd.DataFrame(removal_dot_prod,
                                   index=removal_to_conv.index)[[1]].loc['Start'].values[0]
        removal_effect = 1 - removal_cvr / conversion_rate
        removal_effects_dict[channel] = removal_effect

    return {k: v for k, v in sorted(removal_effects_dict.items())} 

campaign_removal_effect = removal_effects(transition_matrix, base_conversion_rate)

In [15]:
def allocator(removal_effects, unit):
    sum_effect = sum(list(removal_effects.values()))    
    allocation_result = {k: (v / sum_effect) * unit for k, v in sorted(removal_effects.items())}    
    return allocation_result

conversion_unit_attributions = allocator(campaign_removal_effect, 1)

#### RESULT OF CONVERSTION ATTRIBUTION

In [16]:
removal_effects_dict = removal_effects(transition_matrix, base_conversion_rate)
revenue_attributions = allocator(removal_effects_dict, total_value)
print("removal_effects_dict")
print(removal_effects_dict)
print("Conversion Unit Attribution")
print(conversion_unit_attributions)
print("revenue attributions")
print(revenue_attributions)

removal_effects_dict
{'a': 0.8597100706463602, 'b': 0.8414695477652324, 'c': 0.8312865403541212, 'd': 0.8293959429065695}
Conversion Unit Attribution
{'a': 0.2557243707940063, 'b': 0.25029865066347073, 'c': 0.24726967234635122, 'd': 0.24670730619617173}
revenue attributions
{'a': 22486.86682140015, 'b': 22009.761547441634, 'c': 21743.411368104047, 'd': 21693.960263054167}


#### Cost per campaign

In [17]:
total_click_by_campaign = {'a': 0, 'b': 0, 'c': 0, 'd': 0}
for i in range(1, 6):
    click_dict = df[f"click_{i}"].value_counts().to_dict()
    total_click_by_campaign['a'] += click_dict['a']
    total_click_by_campaign['b'] += click_dict['b']
    total_click_by_campaign['c'] += click_dict['c']
    total_click_by_campaign['d'] += click_dict['d']
print(total_click_by_campaign)

{'a': 7364, 'b': 7608, 'c': 7429, 'd': 7381}


In [18]:
cpc = {'a': 7, 'b': 5, 'c': 4, 'd': 2}
cost={}
for campaign in ['a', 'b', 'c', 'd']:
    cost[campaign] = cpc[campaign]*total_click_by_campaign[campaign]
print('Campaign Cost')
print(cost)

Campaign Cost
{'a': 51548, 'b': 38040, 'c': 29716, 'd': 14762}


### ROI per campaign

In [19]:
roi_attributions = {}
for k, v in revenue_attributions.items():
    roi_attributions[k] = (v - cost[k])/cost[k]
print(roi_attributions)

{'a': -0.5637683940909415, 'b': -0.4214047963343419, 'c': -0.2682927928353733, 'd': 0.4695813753593122}


#### Budget Allocation

In [20]:
budget = 1e6
budget_attributions = allocator(removal_effects_dict, budget)

In [21]:
print(budget_attributions)

{'a': 255724.37079400627, 'b': 250298.65066347073, 'c': 247269.67234635123, 'd': 246707.30619617173}


### CONCLUSIONS
- Campaign a is the highest contributor of unit sales
- RoI of each campaign:
   - a ~ 7016 (31.2%)
   - b ~ 11949 (54.3%)
   - c ~ 13827 (63.6%)
   - d ~ 18151 (83.7%)
- For budget allocation, the distribution: a / b / c / d = 25.6 / 25 / 24.7 / 24.7 (in %)